In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from keras.applications.nasnet import NASNetLarge
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, load_model
from keras.layers import *
from keras import backend as K
from keras import optimizers, callbacks
import numpy as np
import pandas as pd
import cv2, h5py

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Model (Inception V3)

In [3]:
batch_size = 128

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'train_aug/',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
        'data2/validation',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

Found 526711 images belonging to 128 classes.
Found 6338 images belonging to 128 classes.


In [7]:
base_model = NASNetLarge(weights=None, include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)
predictions = Dense(128, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [8]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_3
1 stem_conv1
2 stem_bn1
3 activation_527
4 reduction_conv_1_stem_1
5 reduction_bn_1_stem_1
6 activation_528
7 activation_530
8 separable_conv_1_reduction_left1_stem_1
9 separable_conv_1_reduction_1_stem_1
10 separable_conv_1_bn_reduction_left1_stem_1
11 separable_conv_1_bn_reduction_1_stem_1
12 activation_529
13 activation_531
14 separable_conv_2_reduction_left1_stem_1
15 separable_conv_2_reduction_1_stem_1
16 separable_conv_2_bn_reduction_left1_stem_1
17 separable_conv_2_bn_reduction_1_stem_1
18 activation_532
19 reduction_add_1_stem_1
20 separable_conv_1_reduction_right2_stem_1
21 activation_534
22 activation_536
23 separable_conv_1_bn_reduction_right2_stem_1
24 separable_conv_1_reduction_right3_stem_1
25 separable_conv_1_reduction_left4_stem_1
26 activation_533
27 separable_conv_1_bn_reduction_right3_stem_1
28 separable_conv_1_bn_reduction_left4_stem_1
29 separable_conv_2_reduction_right2_stem_1
30 activation_535
31 activation_537
32 reduction_left2_stem_1
33 separable_con

In [9]:
model.save("nasnet.h5")

### Training

In [ ]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("inception-{val_loss:.3f}.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=0)

In [ ]:
!rm -R logs
!ls

In [ ]:
print("Training Progress:")
model_log = model.fit_generator(train_generator, validation_data=validation_generator,
          epochs=5,
          callbacks=[tensorboard, model_checkpoints])

In [ ]:
pd.DataFrame(model_log.history).to_csv("inception-history.csv")

### Fine Tuning

In [ ]:
"""# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)"""

## Evaluation

In [ ]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
%config InlineBackend.figure_format = 'retina'
import itertools, pickle

classes = [str(x) for x in range(0,129)]

In [ ]:
model_test = load_model('best_weights.h5')
Y_test = np.argmax(y_val, axis=1) # Convert one-hot to index
y_pred = model_test.predict(x_val)
y_pred_class = np.argmax(y_pred,axis=1)
cnf_matrix = confusion_matrix(Y_test, y_pred_class)

In [ ]:
print(classification_report(Y_test, y_pred_class, target_names=classes))